In [1]:
import sys
sys.path.append("../../src/models")

from evaluate_model import eval_semantic_segmentation, calc_semantic_segmentation_confusion
from segmentation import SegmentationModel as Model

/opt/conda/envs/py3.6/lib/python3.6/site-packages/cupy/core/fusion.py:659: FutureWarning: cupy.core.fusion is experimental. The interface can change in the future.
  util.experimental('cupy.core.fusion')


In [2]:
from os import path
from PIL import Image
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Load trained model
# Modify the the paths based on your trained model location if needed.
mean = np.load("../../data/dataSplit/mean.npy")
model = Model("../../models/logs/model_iter_14878", mean)

In [4]:
# Load test-set
# Modify the the paths based on your data location if needed.
with open( "../../data/dataSplit/test.txt") as f:
    lines = f.readlines()

test_set = []
for line in lines:
    line = line.rstrip()
    test_set.append(line)

In [5]:
#  Evaluate model

# Modify the the paths based on your raster image and building label location.
image_dir = '../../data/processedBuildingLabels/3band'
label_dir = '../../data/buildingLabelImages'

gt_labels = []
pred_labels = []

for test_data in tqdm(test_set):
    image_path = path.join(image_dir, test_data)
    label_path = path.join(label_dir, test_data)
    
    image = np.array(Image.open(image_path))
    label = np.array(Image.open(label_path))
    
    # Make gt_label, ground-truth building mask
    h, w = label.shape
    gt_label = np.zeros(shape=[h, w], dtype=np.int32) # 0: background
    gt_label[label > 0] = 1 # 1: "building"
    
    # Make pred_label, predicted building mask
    score = model.apply_segmentation(image)
    pred_label = np.argmax(score, axis=0)
    
    gt_labels.append(gt_label)
    pred_labels.append(pred_label)

100%|██████████| 1388/1388 [02:20<00:00,  9.85it/s]


In [6]:
result = eval_semantic_segmentation(pred_labels, gt_labels)
print(result)

{'iou': array([0.95339559, 0.60201456]), 'miou': 0.7777050778994709, 'pixel_accuracy': 0.9564646763810029, 'class_accuracy': array([0.97648055, 0.74886715]), 'mean_class_accuracy': 0.8626738533691427}


In [7]:
building_class = 1
print("IoU for class Building = ", result['iou'][building_class])
print("Accuracy for class Building = ", result['class_accuracy'][building_class])

IoU for class Building =  0.6020145619704008
Accuracy for class Building =  0.7488671526505621


In [8]:
confusion = calc_semantic_segmentation_confusion(pred_labels, gt_labels)
print("confusion:")
print(confusion)

tn = confusion[0][0]
fp = confusion[0][1]
tp = confusion[1][1]
fn = confusion[1][0]

precision = float(tp) / float(tp + fp)
recall       = float(tp) / float(tp + fn)

print()
print("precision: ", precision)
print("recall: ", recall)

confusion:
[[220150859   5302539]
 [  5458996  16278487]]

precision:  0.75429625078993
recall:  0.7488671526505621
